# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [3]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [7]:
gb = GradientBoostingClassifier()
param = {'n_estimators': [10, 150],
         'max_depth': [7, 11, 15],
         'learning_rate': [0.1] 
        }

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,201.941347,1.460429,0.452688,0.033045,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.966786,0.978456,0.971249,0.966757,0.969452,0.970540,0.004307,1
5,255.470594,31.841723,0.192150,0.010738,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.967684,0.975763,0.969452,0.967655,0.969452,0.970001,0.002989,2
3,284.142542,2.254036,0.267871,0.041566,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.962298,0.977558,0.971249,0.969452,0.966757,0.969463,0.005047,3
4,18.678593,0.662283,0.274396,0.012749,0.1,15,10,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.956912,0.968582,0.962264,0.959569,0.961366,0.961738,0.003879,4
2,19.914285,3.678349,0.356619,0.070614,0.1,11,10,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.961400,0.964991,0.958670,0.957772,0.959569,0.960480,0.002555,5


In [8]:
gb = GradientBoostingClassifier()
param = {'n_estimators': [10, 150],
         'max_depth': [7, 11, 15],
         'learning_rate': [0.1] 
        }

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,241.225950,2.356439,0.237419,0.025575,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964093,0.979354,0.968553,0.966757,0.969452,0.969642,0.005189,1
5,222.099829,24.892387,0.180050,0.002888,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.977558,0.968553,0.964061,0.973046,0.969283,0.005431,2
1,175.242591,0.668950,0.372846,0.063801,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965889,0.977558,0.968553,0.964960,0.967655,0.968923,0.004500,3
4,17.870821,0.589249,0.291371,0.013385,0.1,15,10,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.958707,0.962298,0.959569,0.960467,0.965858,0.961380,0.002536,4
2,15.472058,0.239565,0.284069,0.028861,0.1,11,10,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.956014,0.964093,0.953279,0.953279,0.958670,0.957067,0.004043,5
